In [1]:
#cmds maker ( cmds를 위한 형식으로 만들기 위한 json maker. (보기 좋게 cmds로 변환하기 전까지는 json으로 작업)

# actionComment 추가, htmlDate 항목 삭제
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
전자결재 목록 CSV를 JSON 형식으로 변환하는 스크립트
(결재의견 actionComment 추가)
"""

import csv
import json
import re
import sys
from datetime import datetime
from urllib.parse import parse_qs, urlparse

# CSV 필드 크기 제한 증가
csv.field_size_limit(sys.maxsize)

def parse_date_to_unix(date_str):
    """
    날짜 문자열을 유닉스 타임스탬프(밀리초)로 변환
    입력: "2025/10/22 16:27:11" 또는 "2025-10-22 16:27:11"
    """
    if not date_str or date_str.strip() == "":
        return None
    
    # 슬래시를 하이픈으로 변경
    date_str = date_str.replace("/", "-")
    
    try:
        dt = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
        # 밀리초 단위로 변환
        return int(dt.timestamp() * 1000)
    except Exception as e:
        print(f"날짜 변환 오류: {date_str} - {e}")
        return None

def format_html_date(date_str):
    """
    날짜 문자열을 HTML 형식으로 변환 (T를 공백으로)
    입력: "2025/10/22 16:27:11"
    출력: "2025-10-22 16:27:11"
    """
    if not date_str or date_str.strip() == "":
        return ""
    
    # 슬래시를 하이픈으로 변경하고 T를 공백으로
    return date_str.replace("/", "-").replace("T", " ")

def extract_name_from_string(name_str):
    """
    "이름/부서/직급" 형식에서 이름만 추출
    또는 "이름 직급" 형식에서 이름만 추출
    """
    if not name_str or name_str.strip() == "":
        return ""
    
    # "/" 기준으로 분리
    if "/" in name_str:
        return name_str.split("/")[0].strip()
    
    # 공백 기준으로 분리 (첫 번째 공백 전까지)
    if " " in name_str:
        return name_str.split()[0].strip()
    
    return name_str.strip()

def find_user_info(name, users_dict):
    """
    users 딕셔너리에서 이름으로 직위와 부서명 찾기
    """
    user = users_dict.get(name, {})
    return {
        "positionName": user.get("직위명", ""),
        "deptName": user.get("부서명", ""),
        "emailId": user.get("로그인 ID", "")
    }

def parse_content_json(content_str):
    """
    내용 컬럼의 JSON 문자열 파싱
    """
    if not content_str or content_str.strip() == "":
        return {}
    
    try:
        # JSON 문자열 파싱
        content_data = json.loads(content_str)
        return content_data
    except json.JSONDecodeError as e:
        print(f"JSON 파싱 오류: {e}")
        return {}

def load_attachments(attachments_file):
    """
    첨부파일 CSV 파일 로드
    반환: {문서ID: [{파일이름, 저장경로}, ...]}
    """
    attachments_dict = {}
    
    with open(attachments_file, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        for row in reader:
            doc_id = row.get("문서ID", "").strip()
            file_name = row.get("파일이름", "").strip()
            file_path = row.get("저장경로", "").strip()
            
            if doc_id and file_name:
                if doc_id not in attachments_dict:
                    attachments_dict[doc_id] = []
                
                attachments_dict[doc_id].append({
                    "name": file_name,
                    "path": file_path
                })
    
    return attachments_dict

def extract_attachments(content_data, doc_id, attachments_dict):
    """
    첨부파일 정보 추출 - 첨부파일 CSV에서 저장경로 가져오기
    """
    # 첨부파일 CSV에서 해당 문서ID의 첨부파일 정보 가져오기
    if doc_id in attachments_dict:
        return attachments_dict[doc_id]
    
    # 첨부파일 CSV에 없으면 기존 방식으로 처리
    attachments = []
    attach_list = content_data.get("첨부파일", [])
    
    for attach in attach_list:
        if isinstance(attach, dict):
            name = attach.get("파일명", "")
            if name:
                attachments.append({
                    "path": "",
                    "name": name
                })
    
    return attachments

def extract_references(content_data):
    """
    참조문서 정보 추출
    """
    references_list = content_data.get("상단정보", {}).get("참조문서", "")
    
    # 빈 문자열이면 반환
    if not references_list or references_list == []:
        return []
    
    return references_list

def extract_referrers(content_data):
    """
    참조자 정보 추출
    """
    referrers = []
    
    # 상단정보의 참조 필드에서 추출
    refer_str = content_data.get("상단정보", {}).get("참조", "")
    
    if refer_str and refer_str.strip() != "":
        # 여러 참조자가 있을 수 있으므로 쉼표로 분리
        refer_names = [r.strip() for r in refer_str.split(",") if r.strip()]
        
        for refer_full in refer_names:
            name = extract_name_from_string(refer_full)
            if name:
                referrers.append({
                    "name": name,
                    "empNo": "",
                    "deptCode": ""
                })
    
    return referrers

def extract_activities(content_data, drafter_info, users_dict):
    """
    결재라인(activities) 추출
    ①기안자 ②결재자 순서로 구성
    결재의견(actionComment) 포함
    """
    activities = []
    
    # 기안일 (상단정보에서)
    draft_date = content_data.get("상단정보", {}).get("기안일", "")
    draft_html_date = format_html_date(draft_date)
    draft_action_date = parse_date_to_unix(draft_date)
    
    # 결재의견 배열 가져오기
    approval_comments = content_data.get("결재의견", [])
    
    # ① 기안자 추가 (결재의견 배열의 첫 번째 항목)
    drafter_comment = ""
    if len(approval_comments) > 0 and approval_comments[0].get("단계", "") == "기안":
        drafter_comment = approval_comments[0].get("내용", "")
    
    drafter_activity = {
        "positionName": drafter_info["positionName"],
        "deptName": drafter_info["deptName"],
        "actionLogType": "DRAFT",
        # "htmlDate": draft_html_date,
        "name": drafter_info["name"],
        "emailId": drafter_info["emailId"],
        "type": "DRAFT",
        "actionDate": draft_action_date,
        "deptCode": "",
        "actionComment": drafter_comment
    }
    activities.append(drafter_activity)
    
    # ② 결재선 표에서 결재자 추가
    approval_table = content_data.get("결재선_표", [])
    
    # 결재의견 배열에서 기안 이후의 인덱스 추적
    comment_idx = 1  # 0번은 기안자이므로 1번부터 시작
    
    for row in approval_table:
        # 기안 단계는 이미 추가했으므로 건너뛰기
        if row.get("상태", "") == "기안":
            continue
        
        # 결재자 이름 추출
        approver_full = row.get("결재자", "")
        approver_name = extract_name_from_string(approver_full)
        
        # users에서 정보 찾기
        approver_info = find_user_info(approver_name, users_dict)
        
        # 결재일시
        approval_date = row.get("결재일시", "")
        approval_html_date = format_html_date(approval_date)
        approval_action_date = parse_date_to_unix(approval_date)
        
        # 상태에 따라 actionLogType 결정
        state = row.get("상태", "")
        action_type = "APPROVAL"  # 기본값
        
        if state in ["승인", "합의"]:
            action_type = "APPROVAL"
        
        # 해당 결재자의 의견 찾기
        action_comment = ""
        if comment_idx < len(approval_comments):
            action_comment = approval_comments[comment_idx].get("내용", "")
            comment_idx += 1
        
        approver_activity = {
            "positionName": approver_info["positionName"],
            "deptName": approver_info["deptName"],
            "actionLogType": action_type,
            # "htmlDate": approval_html_date,
            "name": approver_name,
            "emailId": approver_info["emailId"],
            "type": action_type,
            "actionDate": approval_action_date,
            "deptCode": "",
            "actionComment": action_comment
        }
        activities.append(approver_activity)
    
    return activities

def extract_doc_body(content_data):
    """
    전자결재 본문 추출
    """
    body_html = content_data.get("본문", {}).get("body_html", "")
    return body_html

def extract_form_name(content_data):
    """
    양식명 추출
    """
    return content_data.get("상단정보", {}).get("양식명", "")

def extract_is_public(content_data):
    """
    문서공개 여부 추출
    """
    is_public_str = content_data.get("상단정보", {}).get("문서공개", "")
    return True if is_public_str == "공개" else False

def load_users(users_file):
    """
    users CSV 파일 로드
    """
    users_dict = {}
    
    with open(users_file, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        for row in reader:
            name = row.get("한글이름", "").strip()
            if name:
                users_dict[name] = {
                    "로그인 ID": row.get("로그인 ID", "").strip(),
                    "부서명": row.get("부서명", "").strip(),
                    "직위명": row.get("직위명", "").strip()
                }
    
    return users_dict

def convert_approval_data(csv_file, users_file, attachments_file, output_file):
    """
    전자결재 CSV를 JSON으로 변환
    """
    # users 파일 로드
    print("users 파일 로드 중...")
    users_dict = load_users(users_file)
    print(f"총 {len(users_dict)}명의 사용자 정보 로드 완료")
    
    # 첨부파일 파일 로드
    print("\n첨부파일 정보 로드 중...")
    attachments_dict = load_attachments(attachments_file)
    print(f"총 {len(attachments_dict)}개 문서의 첨부파일 정보 로드 완료")
    
    # 결과 리스트
    result = []
    
    # CSV 파일 읽기
    print(f"\n{csv_file} 파일 처리 중...")
    with open(csv_file, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        
        row_count = 0
        for row in reader:
            row_count += 1
            
            if row_count % 100 == 0:
                print(f"처리 중... {row_count}건")
            
            try:
                # 기본 정보 추출
                doc_id = row.get("문서ID", "").strip()
                doc_num = row.get("문서번호", "").strip()
                title = row.get("제목", "").strip()
                drafter_str = row.get("기안자", "").strip()
                draft_date = row.get("기안일", "").strip()
                content_str = row.get("내용", "").strip()
                
                # 내용 JSON 파싱
                content_data = parse_content_json(content_str)
                
                # 기안자 정보
                drafter_name = extract_name_from_string(drafter_str)
                drafter_user_info = find_user_info(drafter_name, users_dict)
                
                drafter = {
                    "positionName": drafter_user_info["positionName"],
                    "deptName": drafter_user_info["deptName"],
                    "name": drafter_name,
                    "emailId": drafter_user_info["emailId"],
                    "deptCode": ""
                }
                
                # 날짜 변환
                html_date = format_html_date(draft_date)
                created_at = parse_date_to_unix(draft_date)
                
                # 첨부파일 (첨부파일 CSV에서 가져오기)
                attaches = extract_attachments(content_data, doc_id, attachments_dict)
                
                # 참조문서
                references = extract_references(content_data)
                
                # 참조자
                referrers = extract_referrers(content_data)
                
                # 결재라인 (결재의견 포함)
                activities = extract_activities(content_data, drafter, users_dict)
                
                # 본문
                doc_body = extract_doc_body(content_data)
                
                # 양식명
                form_name = extract_form_name(content_data)
                
                # 문서공개
                is_public = extract_is_public(content_data)
                
                # JSON 객체 생성
                approval_doc = {
                    "sourceId": f"Prefix_{doc_id}",
                    "docNum": doc_num,
                    "references": references,
                    # "htmlDate": html_date,
                    "docType": "DRAFT",
                    "title": title,
                    "attaches": attaches,
                    "drafter": drafter,
                    "createdAt": created_at,
                    "docBody": doc_body,
                    "docStatus": "COMPLETE",
                    "referrers": referrers,
                    "activities": activities,
                    "formName": form_name,
                    "isPublic": is_public
                }
                
                result.append(approval_doc)
                
            except Exception as e:
                print(f"행 {row_count} 처리 중 오류 발생: {e}")
                print(f"문서ID: {row.get('문서ID', '')}")
                continue
    
    # JSON 파일로 저장
    print(f"\n결과 저장 중... 총 {len(result)}건")
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    
    print(f"변환 완료! {output_file} 파일이 생성되었습니다.")
    print(f"총 {len(result)}건의 결재 문서가 변환되었습니다.")

def main():
    """
    메인 함수
    """
    # 파일 경로 설정
    csv_file = "전자결재_목록.csv"
    users_file = "인사정보_부서코드추가.csv"
    attachments_file = "전자결재_첨부파일.csv"
    output_file = "결재목록_변환결과.json"
    
    print("=" * 60)
    print("전자결재 목록 변환 프로그램 (결재의견 포함)")
    print("=" * 60)
    
    convert_approval_data(csv_file, users_file, attachments_file, output_file)
    
    print("\n변환 작업이 완료되었습니다!")

if __name__ == "__main__":
    main()

전자결재 목록 변환 프로그램 (결재의견 포함)
users 파일 로드 중...
총 0명의 사용자 정보 로드 완료

첨부파일 정보 로드 중...
총 3개 문서의 첨부파일 정보 로드 완료

전자결재_목록.csv 파일 처리 중...

결과 저장 중... 총 3건
변환 완료! 결재목록_변환결과.json 파일이 생성되었습니다.
총 3건의 결재 문서가 변환되었습니다.

변환 작업이 완료되었습니다!
